In [4]:
# Import libraries

import pandas as pd
import numpy as np
import os

# 02. Import Data

In [5]:
# Path to main folder
path = r'C:\Users\jmacs\Documents\Career Foundry Tasks\01-2024 Instacart Basket Analysis'

In [6]:
# Importing ords_prods_merge dataframe
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_complete.pkl'))

In [7]:
ords_prods_merge.shape

(32404859, 18)

In [8]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer


# 03. Tasks

In [9]:
# grouping by department id and then aggregating by order number for entire dataframe
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


21 is in the 'missing' department, meaning we can't tell what department the itme belongs to. Perhaps instacart needs to 
re-evaluate their labels for their items?

The next top 2 are 10 'bulk' department and 18 'babies' department. 

In [10]:
# grouping by loyalty_flag and then aggregating by prices
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

Surprisingly, loyal customers on average spend less per order than new and regular customers. 
New customers are spending more than loyal and regular customers. 

In [11]:
# split data into 'user_id' group, 
# apply transform to 'prices' to generate average spending for each user, 
# and create new column 'avg_order_price'

ords_prods_merge['avg_order_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jmacs\AppData\Local\Temp\ipykernel_3468\46790144.py:5: FutureWarning: The provided callable <function mean at 0x00000150231428E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_order_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [12]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_order_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer,6.367797


In [13]:
# setting the flags for high spender and low spender

ords_prods_merge.loc[ords_prods_merge['avg_order_price'] >= 10, 'spending_flag'] = 'High Spender'

In [14]:
ords_prods_merge.loc[ords_prods_merge['avg_order_price'] < 10, 'spending_flag'] = 'Low Spender'

In [15]:
# checking the value counts of spending flag
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [16]:
# limiting output of head function to check columns using df['column']
ords_prods_merge[['user_id', 'spending_flag', 'avg_order_price']].head(60)

,user_id,spending_flag,avg_order_price
0,1,Low Spender,6.367797
1,1,Low Spender,6.367797
2,1,Low Spender,6.367797
3,1,Low Spender,6.367797
4,1,Low Spender,6.367797
5,1,Low Spender,6.367797
6,1,Low Spender,6.367797
7,1,Low Spender,6.367797
8,1,Low Spender,6.367797
9,1,Low Spender,6.367797


In [17]:
# split data into 'user_id' group, 
# apply transform to 'days_since_last_order' to generate median days since last order, 
# and create new column 'avg_days_since_last_order'

ords_prods_merge['avg_days_since_last_order'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.mean)

C:\Users\jmacs\AppData\Local\Temp\ipykernel_3468\2749981782.py:5: FutureWarning: The provided callable <function mean at 0x00000150231428E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_days_since_last_order'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.mean)


In [18]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_order_price,spending_flag,avg_days_since_last_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,Low Spender,20.259259
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer,6.367797,Low Spender,20.259259
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer,6.367797,Low Spender,20.259259
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,Mid-range product,Least busy,Average Orders,10,New customer,6.367797,Low Spender,20.259259
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,Mid-range product,Least busy,Most Orders,10,New customer,6.367797,Low Spender,20.259259


In [19]:
# setting the flags for frequent, regular, non frequent customer

ords_prods_merge.loc[ords_prods_merge['avg_days_since_last_order'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [20]:
ords_prods_merge.loc[(ords_prods_merge['avg_days_since_last_order'] >10) & (ords_prods_merge['avg_days_since_last_order'] <=20), 'frequency_flag'] = 'Regular customer'

In [21]:
ords_prods_merge.loc[ords_prods_merge['avg_days_since_last_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [22]:
# checking the value counts of frequency flag
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        16468582
Regular customer         12521159
Non-frequent customer     3415113
nan                             5
Name: count, dtype: int64

In [23]:
# creating a subset with the rows of missing values from products
df_nan = ords_prods_merge[ords_prods_merge['avg_days_since_last_order'].isnull() == True]

In [24]:
df_nan # I find this odd that there are 5 nulls and they all belong to this one user, they may have never placed another order again?

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_order_price,spending_flag,avg_days_since_last_order,frequency_flag
13645692,895835,159838,1,0,17,NaN,10749,3,0,Organic Red Bell Pepper,...,5.7,Mid-range product,Busiest day,Average Orders,1,New customer,7.42,Low Spender,NaN,nan
17251990,895835,159838,1,0,17,NaN,33401,6,0,Goat Cheese Crumbles,...,14.8,Mid-range product,Busiest day,Average Orders,1,New customer,7.42,Low Spender,NaN,nan
17622767,895835,159838,1,0,17,NaN,23695,2,0,California Veggie Burger,...,4.7,Low-range product,Busiest day,Average Orders,1,New customer,7.42,Low Spender,NaN,nan
24138593,895835,159838,1,0,17,NaN,21334,5,0,Organic Peeled Garlic,...,10.2,Mid-range product,Busiest day,Average Orders,1,New customer,7.42,Low Spender,NaN,nan
25880002,895835,159838,1,0,17,NaN,22198,1,0,4X Ultra Concentrated Natural Laundry Detergen...,...,1.7,Low-range product,Busiest day,Average Orders,1,New customer,7.42,Low Spender,NaN,nan


In [25]:
# checking dataframe for missing values
ords_prods_merge.isnull().sum()

order_id                           0
user_id                            0
order_number                       0
orders_day_of_week                 0
order_hour_of_day                  0
days_since_last_order        2076096
product_id                         0
add_to_cart_order                  0
reordered                          0
product_name                       0
aisle_id                           0
department_id                      0
prices                             0
price_range_loc                    0
busiest_day                        0
busiest_period_of_day              0
max_order                          0
loyalty_flag                       0
avg_order_price                    0
spending_flag                      0
avg_days_since_last_order          5
frequency_flag                     0
dtype: int64

In [26]:
# limiting output of head function to check columns using df['column']
ords_prods_merge[['user_id', 'frequency_flag', 'avg_days_since_last_order']].head(60)

,user_id,frequency_flag,avg_days_since_last_order
0,1,Non-frequent customer,20.259259
1,1,Non-frequent customer,20.259259
2,1,Non-frequent customer,20.259259
3,1,Non-frequent customer,20.259259
4,1,Non-frequent customer,20.259259
5,1,Non-frequent customer,20.259259
6,1,Non-frequent customer,20.259259
7,1,Non-frequent customer,20.259259
8,1,Non-frequent customer,20.259259
9,1,Non-frequent customer,20.259259


In [27]:
# exporting orders_products_merge in pickle format
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_complete.pkl'))